In [1]:
import random
import torch
import numpy as np
from tqdm import tqdm
import time
import logging
from sklearn.model_selection import StratifiedKFold
import os
import pandas as pd
from torch.utils.data import (DataLoader,RandomSampler,SequentialSampler,TensorDataset)
from sklearn.metrics import accuracy_score, f1_score
from transformers import BertModel, BertConfig
from transformers import AdamW
from transformers import BertTokenizer
from transformers.optimization import get_linear_schedule_with_warmup
import torch.nn as nn
import torch.nn.functional as F
from collections import Counter
from torch.cuda.amp import autocast, GradScaler
from torch.optim.optimizer import Optimizer
import math
import Levenshtein
from pypinyin import lazy_pinyin
import jieba
import argparse


In [51]:
###参数设置
parser = argparse.ArgumentParser()
parser.add_argument('--random_seed',type=int,help='随机种子',default = 78)
parser.add_argument('--file_name',type=str,default = 'baseline1')
parser.add_argument('--file_path',type=str,default='./log/log_modeltest.txt')
parser.add_argument('--max_seq_length',type=int,default = 60)
parser.add_argument('--learning_rate',type=float,default = 5e-5)
parser.add_argument('--num_epochs',type=int,default = 8)
parser.add_argument('--batch_size',type=int,default = 100)
parser.add_argument('--patience',type=int,default = 5)
parser.add_argument('--train_input',type=str,default = './data/train/')
parser.add_argument('--test_input',type=str,default = './data/test/')
parser.add_argument('--model_name_or_path',type=str,default = './pretrain_models/ernie-gram/')
args = parser.parse_args(args=[])


In [3]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ['PYTHONHASHSEED']= str(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(args.random_seed)

def get_logger(filename,verbosity=1,name=None):
    level_dict = {0:logging.DEBUG,1:logging.INFO,2:logging.WARNING}
    formatter = logging.Formatter(
        "[%(asctime)s][%(filename)s][line:%(lineno)d][%(levelname)s]## %(message)s"
    )
    logger = logging.getLogger(name)
    logger.setLevel(level_dict[verbosity])
    timestamp = time.strftime("%Y.%m.%d_%H.%M.%S",time.localtime())
    
    fh = logging.FileHandler(filename,"w")
    fh.setFormatter(formatter)
    logger.addHandler(fh)
    
    sh = logging.StreamHandler()
    sh.setFormatter(formatter)
    logger.addHandler(sh)
    
    return logger
logger = get_logger(args.file_path)

In [33]:
class InputExample(object):
    def __init__(self,s1,s2,label=None):
        self.s1 = s1
        self.s2 = s2
        self.label = label

class InputFeatures(object):
    def __init__(self,choices_features,label):
        _,input_ids,input_mask,segment_ids = choices_features[0]
        self.choices_features = {
            'input_ids':input_ids,
            'input_mask':input_mask,
            'segment_ids':segment_ids
        }
        self.label = label
        
def read_data(file_name):
    examples = []
    with open(file_name,'r',encoding='utf-8') as f:
        for line in f.readlines():
            line = line.strip()
            if line:
                line =line.split('\t')
                examples.append(InputExample(s1=line[0],s2=line[1],label=line[2]) if len(line)==3 else None)
    return examples

def read_examples(dir,split='train'):
    examples = []
    for path in os.listdir(dir):
        if split=='train':
            for file_name in os.listdir(dir+path):
                example = read_data(os.path.join(dir+path,file_name))
                examples.extend(example)
    return examples

def _truncate_seq_pair(tokens_a,tokens_b,max_length):
    while True:
        total_length = len(tokens_a)+len(tokens_b)
        if total_length<=max_length:
            break
        if len(tokens_a)>len(tokens_b):
            tokens_a.pop()    
        else:
            tokens_b.pop()

def convert_examples_to_features(examples,tokenizer,max_seq_length,is_training):
    features = []
    for example_index,example in enumerate(examples):
        
        s1 = tokenizer.tokenize(example.s1)
        s2 = tokenizer.tokenize(example.s2)
        _truncate_seq_pair(s1, s2, max_seq_length)
        
        choices_features = []
        
        tokens = ["[CLS]"] + s1 +["[SEP]"] + s2 + ["[SEP]"]
        segment_ids = [0]*(len(s1)+2)+[1]*(len(s2)+1)
        input_ids = tokenizer.convert_tokens_to_ids(tokens)
        input_mask = [1]*len(tokens)
        ## mask has 1 for real token 0 for padding token
        
        padding_length = max_seq_length-len(tokens)+3
        input_ids +=([0]*padding_length)
        input_mask +=([0]*padding_length)
        segment_ids +=([0]*padding_length)
        choices_features.append((tokens,input_ids,input_mask,segment_ids))
        
        label = example.label
        if example_index < 1 and is_training:
            logger.info("*** Example ***")
            logger.info("idx:{}".format(example_index))
            logger.info("tokens:{}".format(' '.join(tokens).replace('\u2581','_')))
            logger.info("input_ids:{}".format(' '.join(map(str,input_ids))))
            logger.info("input_mask:{}".format(len(input_mask)))
            logger.info("segment_ids:{}".format(len(segment_ids)))
            logger.info("label:{}".format(label))
        
        features.append(
            InputFeatures(
                choices_features = choices_features,
                label = label
            )
        )
        
    return features
    
def select_field(features,field):
    return[
        feature.choices_features[field] for feature in features
    ]

In [34]:
class NeuralNet(nn.Module):
    def __init__(self,model_name_or_path,hidden_size=768,num_class=2):
        super(NeuralNet,self).__init__()
        
        self.config = BertConfig.from_pretrained(model_name_or_path,num_lables=num_class)
        self.config.output_hidden_states =True
        self.bert = BertModel.from_pretrained(model_name_or_path,config=self.config)
        for param in self.bert.parameters():
            param.requires_grad = True    #如果为False ,则冻结预训练模型的权重
        self.weights = nn.Parameter(torch.rand(13,1))
        self.fc = nn.Linear(hidden_size*2,num_class)
        self.dropouts = nn.ModuleList([
            nn.Dropout(0.2) for _ in range(5)
        ])
    
    def forward(self,input_ids,input_mask,segment_ids,y=None,loss_fn=None):
        output = self.bert(input_ids,token_type_ids=segment_ids,attention_mask=input_mask)
        last_hidden = output.last_hidden_state # 最后一层输出的隐藏状态 batch_size * sequence_length * hidden_size
        all_hidden_states = output.hidden_states 
        batch_size = input_ids.shape[0]
        #####补充   output 为tuple类型，共四个部分，last_hidden_state ，pooler_output （序列第一个token的最后一层隐藏状态），
        ######      hiddden_states 是一个元组（第一个元素是embedding，其余元素是各层的输出）
        ######      attentions 元素是每一层的注意力权重，用于计算self.attention heads 的加权平均值
        
        ht_cls = torch.cat(all_hidden_states)[:,:1,:].view(13,batch_size,1,768)  # 每个样本的每一层的ht_cls
        atten = torch.sum(ht_cls*self.weights.view(13,1,1,1),dim=[1,3])#缩减维度
        atten = F.softmax(atten.view(-1),dim=0)
        feature = torch.sum(ht_cls*atten.view(13,1,1,1),dim=[0,2])
        f = torch.mean(last_hidden,1)
        feature = torch.cat((feature,f),1)  #batch_size*(hidden_size*2)
        
        for i , dropout in enumerate(self.dropouts):
            if i==0:
                h = self.fc(dropout(feature))
                if loss_fn is not None:
                    loss = loss_fn(h,y)
            else:
                hi = self.fc(dropout(feature))
                h = h + hi
                if loss_fn is not None:
                    loss += loss_fn(hi,y)
        
        if loss_fn is not None:
            return h/len(self.dropouts), loss/len(self.dropouts)
        return h/len(self,dropouts)
        
    

In [35]:
class RAdam(Optimizer):
    def __init__(self,params,lr=1e-3,betas=(0.9,0.999),eps=1e-8,weight_decay=0):
        if not 0.0 <= lr:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if not 0.0 <= eps:
            raise ValueError("Invalid epsilon value: {}".format(eps))
        if not 0.0 <= betas[0] < 1.0:
            raise ValueError("Invalid beta parameter at index 0: {}".format(betas[0]))
        if not 0.0 <= betas[1] < 1.0:
            raise ValueError("Invalid beta parameter at index 1: {}".format(betas[1]))

        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay)
        self.buffer = [[None, None, None] for ind in range(10)]
        super(RAdam, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(RAdam, self).__setstate__(state)

    def step(self, closure=None):

        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:

            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data.float()
                if grad.is_sparse:
                    raise RuntimeError('RAdam does not support sparse gradients')

                p_data_fp32 = p.data.float()

                state = self.state[p]

                if len(state) == 0:
                    state['step'] = 0
                    state['exp_avg'] = torch.zeros_like(p_data_fp32)
                    state['exp_avg_sq'] = torch.zeros_like(p_data_fp32)
                else:
                    state['exp_avg'] = state['exp_avg'].type_as(p_data_fp32)
                    state['exp_avg_sq'] = state['exp_avg_sq'].type_as(p_data_fp32)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']

                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)
                exp_avg.mul_(beta1).add_(1 - beta1, grad)

                state['step'] += 1
                buffered = self.buffer[int(state['step'] % 10)]
                if state['step'] == buffered[0]:
                    N_sma, step_size = buffered[1], buffered[2]
                else:
                    buffered[0] = state['step']
                    beta2_t = beta2 ** state['step']
                    N_sma_max = 2 / (1 - beta2) - 1
                    N_sma = N_sma_max - 2 * state['step'] * beta2_t / (1 - beta2_t)
                    buffered[1] = N_sma

                    # more conservative since it's an approximated value
                    if N_sma >= 5:
                        step_size = math.sqrt(
                            (1 - beta2_t) * (N_sma - 4) / (N_sma_max - 4) * (N_sma - 2) / N_sma * N_sma_max / (
                            N_sma_max - 2)) / (1 - beta1 ** state['step'])
                    else:
                        step_size = 1.0 / (1 - beta1 ** state['step'])
                    buffered[2] = step_size

                if group['weight_decay'] != 0:
                    p_data_fp32.add_(-group['weight_decay'] * group['lr'], p_data_fp32)

                # more conservative since it's an approximated value
                if N_sma >= 5:
                    denom = exp_avg_sq.sqrt().add_(group['eps'])
                    p_data_fp32.addcdiv_(-step_size * group['lr'], exp_avg, denom)
                else:
                    p_data_fp32.add_(-step_size * group['lr'], exp_avg)

                p.data.copy_(p_data_fp32)

        return loss
        

In [36]:
class focal_loss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2, num_classes = 2, size_average=True):
        """
        focal_loss损失函数, -α(1-yi)**γ *ce_loss(xi,yi)
        步骤详细的实现了 focal_loss损失函数.
        :param alpha:   阿尔法α,类别权重.      当α是列表时,为各类别权重,当α为常数时,类别权重为[α, 1-α, 1-α, ....],常用于 目标检测算法中抑制背景类 , retainnet中设置为0.25
        :param gamma:   伽马γ,难易样本调节参数. retainnet中设置为2
        :param num_classes:     类别数量
        :param size_average:    损失计算方式,默认取均值
        """
        super(focal_loss,self).__init__()
        self.size_average = size_average
        if isinstance(alpha,list):
            assert len(alpha)==num_classes   # α可以以list方式输入,size:[num_classes] 用于对不同类别精细地赋予权重
            # print(" --- Focal_loss alpha = {}, 将对每一类权重进行精细化赋值 --- ".format(alpha))
            self.alpha = torch.Tensor(alpha)
        else:
            assert alpha<1   #如果α为一个常数,则降低第一类的影响,在目标检测中为第一类
            # print(" --- Focal_loss alpha = {} ,将对背景类进行衰减,请在目标检测任务中使用 --- ".format(alpha))
            self.alpha = torch.zeros(num_classes)
            self.alpha[0] += alpha
            self.alpha[1:] += (1-alpha) # α 最终为 [ α, 1-α, 1-α, 1-α, 1-α, ...] size:[num_classes]

        self.gamma = gamma

    def forward(self, preds, labels):
        """
        focal_loss损失计算
        :param preds:   预测类别. size:[B,N,C] or [B,C]    分别对应与检测与分类任务, B 批次, N检测框数, C类别数
        :param labels:  实际类别. size:[B,N] or [B]
        :return:
        """
        # assert preds.dim()==2 and labels.dim()==1
        preds = preds.view(-1,preds.size(-1))
        self.alpha = self.alpha.to(preds.device)
        preds_logsoft = F.log_softmax(preds, dim=1) # log_softmax
        preds_softmax = torch.exp(preds_logsoft)    # softmax

        preds_softmax = preds_softmax.gather(1,labels.view(-1,1))   # 这部分实现nll_loss ( crossempty = log_softmax + nll )
        preds_logsoft = preds_logsoft.gather(1,labels.view(-1,1))
        self.alpha = self.alpha.gather(0,labels.view(-1))
        loss = -torch.mul(torch.pow((1-preds_softmax), self.gamma), preds_logsoft)  # torch.pow((1-preds_softmax), self.gamma) 为focal loss中 (1-pt)**γ

      #  loss = torch.mul(self.alpha, loss.t())
        if self.size_average:
            loss = loss.mean()
        else:
            loss = loss.sum()
        return loss


class FGM():
    def __init__(self, model):
        self.model = model
        self.backup = {}

    def attack(self, epsilon=1., emb_name='bert.embeddings.'):
        # emb_name这个参数要换成你模型中embedding的参数名
        for name, param in self.model.named_parameters():
            if param.requires_grad and emb_name in name:
                self.backup[name] = param.data.clone()
                norm = torch.norm(param.grad)
                if norm != 0 and not torch.isnan(norm):
                    r_at = epsilon * param.grad / norm
                    param.data.add_(r_at)

    def restore(self, emb_name='bert.embeddings.'):
        # emb_name这个参数要换成你模型中embedding的参数名
        for name, param in self.model.named_parameters():
            if param.requires_grad and emb_name in name: 
                assert name in self.backup
                param.data = self.backup[name]
        self.backup = {}


class PGD():
    def __init__(self, model):
        self.model = model
        self.emb_backup = {}
        self.grad_backup = {}

    def attack(self, epsilon=1., alpha=0.3, emb_name='bert.embeddings.', is_first_attack=False):
        # emb_name这个参数要换成你模型中embedding的参数名
        for name, param in self.model.named_parameters():
            if param.requires_grad and emb_name in name:
                if is_first_attack:
                    self.emb_backup[name] = param.data.clone()
                norm = torch.norm(param.grad)
                if norm != 0 and not torch.isnan(norm):
                    r_at = alpha * param.grad / norm
                    param.data.add_(r_at)
                    param.data = self.project(name, param.data, epsilon)

    def restore(self, emb_name='bert.embeddings.'):
        # emb_name这个参数要换成你模型中embedding的参数名
        for name, param in self.model.named_parameters():
            if param.requires_grad and emb_name in name: 
                assert name in self.emb_backup
                param.data = self.emb_backup[name]
        self.emb_backup = {}

    def project(self, param_name, param_data, epsilon):
        r = param_data - self.emb_backup[param_name]
        if torch.norm(r) > epsilon:
            r = epsilon * r / torch.norm(r)
        return self.emb_backup[param_name] + r

    def backup_grad(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                # 不对最后的 bert.pooler 层和 linear1 层做对抗训练
                if 'encoder' in name or 'bert.embeddings.' in name:
                    self.grad_backup[name] = param.grad.clone()

    def restore_grad(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                if 'encoder' in name or 'bert.embeddings.' in name:
                    param.grad = self.grad_backup[name]

In [37]:
def metric(y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='macro')
    return acc, f1

def set_lr(optimizer, value):
    for p in optimizer.param_groups:
        p['lr'] = value

#指数加权平均
class EMA():
    def __init__(self, model, decay):
        self.model = model
        self.decay = decay
        self.shadow = {}
        self.backup = {}

    def register(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                self.shadow[name] = param.data.clone()

    def update(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                assert name in self.shadow
                new_average = (1.0 - self.decay) * param.data + self.decay * self.shadow[name]
                self.shadow[name] = new_average.clone()

    def apply_shadow(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                assert name in self.shadow
                self.backup[name] = param.data
                param.data = self.shadow[name]

    def restore(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                assert name in self.backup
                param.data = self.backup[name]
        self.backup = {}


In [42]:
# 加载数据
tokenizer = BertTokenizer.from_pretrained(args.model_name_or_path, do_lower_case=True)
train_examples = read_examples(args.train_input, split='train')
train_features = convert_examples_to_features(
    train_examples, tokenizer, args.max_seq_length,True)

all_input_ids = np.array(select_field(train_features, 'input_ids'))
logger.info('shape: {}'.format(all_input_ids.shape))
all_input_mask = np.array(select_field(train_features, 'input_mask'))
all_segment_ids = np.array(select_field(train_features, 'segment_ids'))
all_label = np.array([int(f.label) for f in train_features])
logger.info(Counter(all_label))


test_examples = read_examples(args.test_input, split='test')
test_features = convert_examples_to_features(
    test_examples, tokenizer, args.max_seq_length, True)
test_input_ids = torch.tensor(select_field(test_features, 'input_ids'), dtype=torch.long)
test_input_mask = torch.tensor(select_field(test_features, 'input_mask'), dtype=torch.long)
test_segment_ids = torch.tensor(select_field(test_features, 'segment_ids'), dtype=torch.long)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=args.random_seed)
oof_train = np.zeros((len(train_examples), 2), dtype=np.float32)
oof_test = np.zeros((len(test_examples), 2), dtype=np.float32)

[2021-11-22 10:52:13,993][2469872708.py][line:70][INFO]## *** Example ***
[2021-11-22 10:52:13,994][2469872708.py][line:71][INFO]## idx:0
[2021-11-22 10:52:13,994][2469872708.py][line:72][INFO]## tokens:[CLS] 喜 欢 打 篮 球 的 男 生 喜 欢 什 么 样 的 女 生 [SEP] 爱 打 篮 球 的 男 生 喜 欢 什 么 样 的 女 生 [SEP]
[2021-11-22 10:52:13,995][2469872708.py][line:73][INFO]## input_ids:1 692 811 445 2001 497 5 654 21 692 811 614 356 314 5 291 21 2 329 445 2001 497 5 654 21 692 811 614 356 314 5 291 21 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
[2021-11-22 10:52:13,996][2469872708.py][line:74][INFO]## input_mask:63
[2021-11-22 10:52:13,996][2469872708.py][line:75][INFO]## segment_ids:63
[2021-11-22 10:52:13,997][2469872708.py][line:76][INFO]## label:1
[2021-11-22 10:54:12,337][1912096065.py][line:8][INFO]## shape: (698577, 63)
[2021-11-22 10:54:18,288][1912096065.py][line:12][INFO]## Counter({0: 355865, 1: 342712})


In [ ]:
batch_size = args.batch_size
for fold, (train_index, valid_index) in enumerate(skf.split(all_label, all_label)):
    logger.info('================     fold {}        ==============='.format(fold))

    # 处理模型输入数据
    train_input_ids = torch.tensor(all_input_ids[train_index], dtype=torch.long)
    train_input_mask = torch.tensor(all_input_mask[train_index], dtype=torch.long)
    train_segment_ids = torch.tensor(all_segment_ids[train_index], dtype=torch.long)
    train_label = torch.tensor(all_label[train_index], dtype=torch.long)

    valid_input_ids = torch.tensor(all_input_ids[valid_index], dtype=torch.long)
    valid_input_mask = torch.tensor(all_input_mask[valid_index], dtype=torch.long)
    valid_segment_ids = torch.tensor(all_segment_ids[valid_index], dtype=torch.long)
    valid_label = torch.tensor(all_label[valid_index], dtype=torch.long)

    train = torch.utils.data.TensorDataset(train_input_ids, train_input_mask, train_segment_ids, train_label)
    valid = torch.utils.data.TensorDataset(valid_input_ids, valid_input_mask, valid_segment_ids, valid_label)
    test = torch.utils.data.TensorDataset(test_input_ids, test_input_mask, test_segment_ids)

    train_loader = torch.utils.data.DataLoader(train, batch_size=args.batch_size, shuffle=True)
    valid_loader = torch.utils.data.DataLoader(valid, batch_size=args.batch_size, shuffle=False)
    test_loader = torch.utils.data.DataLoader(test, batch_size=args.batch_size, shuffle=False)

    model = NeuralNet(args.model_name_or_path).cuda()
    model.cuda()
    # model = nn.DataParallel(model, device_ids=[0, 1])
    loss_fn = torch.nn.CrossEntropyLoss()
#     loss_fn = focal_loss()
    
#     fgm = FGM(model)
    


    # 优化器定义
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}]

    optimizer = RAdam(optimizer_grouped_parameters, lr=args.learning_rate, eps=1e-6)
    total_steps = args.num_epochs * len(train_loader)
    # scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=len(train_loader)*2, num_training_steps=total_steps)
    scaler = GradScaler()

    best_f1 = 0.
    valid_best = np.zeros((valid_label.size(0), 2))

    early_stop = 0
    ema = EMA(model, 0.999)
    ema.register()
    for epoch in range(args.num_epochs):
        train_loss = 0.
        lr_list = []
        # if epoch > 2:
        #     set_lr(optimizer, 2e-5)
        model.train()
        for batch in tqdm(train_loader):
            optimizer.zero_grad()
            batch = tuple(t.cuda() for t in batch)
            x_ids, x_mask, x_sids, y_truth = batch
            with autocast():
                y_pred, loss = model(x_ids, x_mask, x_sids, y=y_truth, loss_fn=loss_fn)
                        
            scaler.scale(loss.mean()).backward()
            #加入对抗训练
#             fgm.attack()
#             y_pred, loss_sum = model(x_ids, x_mask, x_sids, y=y_truth, loss_fn=loss_fn)
#             loss_sum.backward()
#             fgm.restore()
            scaler.step(optimizer)
            scale = scaler.get_scale()
            scaler.update()
            ema.update()
            # skip_lr_sched = (scale != scaler.get_scale())
            # if not skip_lr_sched:
            #     scheduler.step()
            train_loss += loss.mean().item() / len(train_loader)

        ema.apply_shadow()
        model.eval()
        val_loss = 0.
        valid_preds_fold = np.zeros((valid_label.size(0), 2))
        with torch.no_grad():
            for i, batch in tqdm(enumerate(valid_loader)):
                batch = tuple(t.cuda() for t in batch)
                x_ids, x_mask, x_sids, y_truth = batch
                with autocast():
                    y_pred, loss = model(x_ids, x_mask, x_sids, y_truth, loss_fn)
                    y_pred = y_pred.detach()
                    val_loss += loss.mean().item() / len(valid_loader)
                valid_preds_fold[i * batch_size:(i + 1) * batch_size] = F.softmax(y_pred, dim=1).cpu().numpy()
        acc, f1 = metric(all_label[valid_index], np.argmax(valid_preds_fold, axis=1))
        if best_f1 < f1:
            early_stop = 0
            best_f1 = f1
            valid_best = valid_preds_fold
            torch.save(model.state_dict(), './model_save/ernie_' + args.file_name + '_{}.bin'.format(fold))
        else:
            early_stop += 1
        logger.info(
            'epoch: %d, train loss: %.8f, valid loss: %.8f, acc: %.8f, f1: %.8f, best_f1: %.8f\n' %
            (epoch, train_loss, val_loss, acc, f1, best_f1))
        torch.cuda.empty_cache()  # 每个epoch结束之后清空显存，防止显存不足

        # 检测早停
        if early_stop >= args.patience:
            break

    # 得到一折模型对测试集的预测结果
    model.load_state_dict(torch.load('./model_save/ernie_' + args.file_name + '_{}.bin'.format(fold)))
    test_preds_fold = np.zeros((len(test_examples), 2))
    model.eval()
    with torch.no_grad():
        for i, batch in tqdm(enumerate(test_loader)):
            batch = tuple(t.cuda() for t in batch)
            x_ids, x_mask, x_sids = batch
            with autocast():
                y_pred = model(x_ids, x_mask, x_sids).detach()
            test_preds_fold[i * batch_size:(i + 1) * batch_size] = F.softmax(y_pred, dim=1).cpu().numpy()

    oof_train[valid_index] = valid_best
    acc, f1 = metric(all_label[valid_index], np.argmax(valid_best, axis=1))
    logger.info('epoch: best, acc: %.8f, f1: %.8f, best_f1: %.8f\n' %
                (acc, f1, best_f1))
    oof_test += test_preds_fold / 5


[2021-11-22 12:56:57,010][2529106677.py][line:3][INFO]## ================     fold 0        ===============
 13%|█████████████▎                                                                                     | 752/5589 [01:58<12:35,  6.40it/s]

In [ ]:
# 保存概率文件
np.savetxt('./submit/train_prob/train_bert_' + args.file_name + '.txt', oof_train)
np.savetxt('./submit/test_prob/test_bert_' + args.file_name + '.txt', oof_test)
acc, f1 = metric(all_label, np.argmax(oof_train, axis=1))
logger.info('epoch: best, acc: %.8f, f1: %.8f \n' % (acc, f1))

analysis = pd.DataFrame()
analysis['s1'] = [line.s1 for line in train_examples]
analysis['s2'] = [line.s2 for line in train_examples]
analysis['label'] = [line.label for line in train_examples]
analysis['pred'] = np.argmax(oof_train, axis=1).tolist()
analysis[analysis['label'] != analysis['pred']].to_csv('analysis_{}.csv'.format(f1), index=False)


In [ ]:
def prob_postprocess(y_pred):
    prior = np.array([0.6903327690476333, 0.3096672309523667]) # 训练集 oppo正负样本比例
    
#     prior = np.array([0.52618167,0.47381833]) 
    y_pred_uncertainty = -(y_pred * np.log(y_pred)).sum(1) / np.log(2)

    threshold = 0.95
    y_pred_confident = y_pred[y_pred_uncertainty < threshold]
    y_pred_unconfident = y_pred[y_pred_uncertainty >= threshold]

    right, alpha, iters = 0, 1, 1
    post = []
    for i, y in enumerate(y_pred_unconfident):
        Y = np.concatenate([y_pred_confident, y[None]], axis=0)
        for j in range(iters):
            Y = Y ** alpha
            Y /= Y.sum(axis=0, keepdims=True)
            Y *= prior[None]
            Y /= Y.sum(axis=1, keepdims=True)
        y = Y[-1]
        post.append(y.tolist())

    post = np.array(post)
    y_pred[y_pred_uncertainty >= threshold] = post

    return y_pred

In [ ]:
# 后处理
oof_test = prob_postprocess(oof_test)
y_preds = np.argmax(oof_test, axis=1)
logger.info(Counter(y_preds))

with open('./output/predict_result_{}.csv'.format(f1), 'w', encoding="utf-8") as f:
    for y_pred in y_preds:
        f.write(str(y_pred) + "\n")


def compare_pinyin(s1, s2):
    s1_pinyin = ""
    s2_pinyin = ""
    for w in jieba.cut(s1):
        s1_pinyin += ''.join(lazy_pinyin(w))
    for w in jieba.cut(s2):
        s2_pinyin += ''.join(lazy_pinyin(w))
    return s1_pinyin == s2_pinyin


def postprocess(data, pred):
    post = []
    for line, lable in tqdm(zip(data, pred)):
        # r1 = correct(line.s1, line.s2)  # 339
        r2 = compare_pinyin(line.s1, line.s2)  # 339
        if r2:
            post.append(1)
        else:
            post.append(lable)
    post = np.array(post)
    print(np.count_nonzero(post != pred))
    return post

post = postprocess(test_examples, y_preds)

with open('./output/post_predict_result_{}.csv'.format(f1), 'w', encoding="utf-8") as f:
    for y_pred in post:
        f.write(str(y_pred) + "\n")